In [141]:
from __future__ import division, print_function

In [142]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
import os
import json

In [143]:
# os.chdir('./bucket')
os.getcwd()

'/Users/hmourit/Documents/0project/bucket'

In [144]:
results = json.load(open('./results/results.json', 'r'))
results = pd.io.json.json_normalize(results)
results['target'] = results.target.fillna('drug')
results

,classifier,cross_val.n_folds,cross_val.shuffle,cross_val.type,results.accuracy.mean,results.accuracy.std,results.time,split.n_iter,split.test_size,split.type,target
0,SVC,20,True,StratifiedKFold,0.414286,0.162255,NaN,10,0.1,StratifiedShuffleSplit,drug
1,SVC,10,True,StratifiedKFold,0.435714,0.087190,NaN,10,0.1,StratifiedShuffleSplit,drug
2,SVC,10,True,StratifiedKFold,0.378049,0.045303,NaN,10,0.3,StratifiedShuffleSplit,drug
3,SVC,10,True,StratifiedKFold,0.752500,0.077015,1:01:31,10,0.3,StratifiedShuffleSplit,stress
4,SVC,10,True,StratifiedKFold,0.742500,0.062300,1:08:53,10,0.3,StratifiedShuffleSplit,stress
5,SVC,10,True,StratifiedKFold,0.777500,0.069327,1:04:04,10,0.3,StratifiedShuffleSplit,stress
6,SVC,10,True,StratifiedKFold,0.762500,0.053910,0:57:07,10,0.3,StratifiedShuffleSplit,stress
7,SVC,10,True,StratifiedKFold,0.780000,0.103494,0:19:59,10,0.1,StratifiedShuffleSplit,stress
8,SVC,10,True,StratifiedKFold,0.824444,0.106782,0:33:50,30,0.1,StratifiedShuffleSplit,stress
9,EN,10,NaN,NaN,0.350000,0.021429,0:14:23,10,0.1,StratifiedShuffleSplit,drug


In [145]:
temp = results[results['split.test_size'] == 0.1][results['classifier'] == 'EN']
temp = temp[['split.n_iter', 'cross_val.n_folds', 'results.accuracy.mean', 'results.accuracy.std', 'target']]
temp[temp['target'] == 'drug']

/Users/hmourit/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,split.n_iter,cross_val.n_folds,results.accuracy.mean,results.accuracy.std,target
9,10,10,0.350000,0.021429,drug
13,10,10,0.364286,0.038465,drug
14,30,10,0.330952,0.046960,drug


In [146]:
temp = results[results['classifier'] == 'SVC']
temp = temp[['split.test_size', 'split.n_iter', 'cross_val.n_folds', 'results.accuracy.mean', 'results.accuracy.std', 'target']]
temp[temp['target'] == 'drug']

,split.test_size,split.n_iter,cross_val.n_folds,results.accuracy.mean,results.accuracy.std,target
0,0.1,10,20,0.414286,0.162255,drug
1,0.1,10,10,0.435714,0.087190,drug
2,0.3,10,10,0.378049,0.045303,drug
10,0.1,10,20,0.364286,0.129560,drug
19,0.1,30,10,0.373810,0.102159,drug


In [ ]:
def combine_means():
    pass

def combine_stds():
    pass

In [161]:
# keys = ['classifier', 'cross_val.n_folds', 'split.test_size', 'target']
keys = ['classifier', 'target']

In [165]:
# def combine_means_and_stds(results, keys):
results['n*mu'] = results['split.n_iter'] * results['results.accuracy.mean']
results['ns2m2'] = results['split.n_iter'] * \
    (results['results.accuracy.std']**2 + results['results.accuracy.mean']**2)
aggregated = results.groupby(keys).agg(np.sum)
aggregated['mean'] = aggregated['n*mu'] / aggregated['split.n_iter']
aggregated['variance'] = (aggregated['ns2m2'] - aggregated['mean']**2 * aggregated['split.n_iter']) / aggregated['split.n_iter']
aggregated['std'] = np.sqrt(aggregated['variance'])
aggregated[['split.n_iter', 'mean', 'std']]

split.n_iter      mean       std
classifier target                                  
EN         drug              60  0.338995  0.042332
           stress            50  0.377333  0.080675
SVC        drug              70  0.387680  0.112319
           stress           100  0.801500  0.096885

In [151]:
combine_means_and_stds(results, keys)

,classifier,cross_val.n_folds,cross_val.shuffle,cross_val.type,results.accuracy.mean,results.accuracy.std,results.time,split.n_iter,split.test_size,split.type,target,n*mu,combined_mean
0,SVC,20,True,StratifiedKFold,0.414286,0.162255,NaN,10,0.1,StratifiedShuffleSplit,drug,4.142857,0.389286
1,SVC,10,True,StratifiedKFold,0.435714,0.087190,NaN,10,0.1,StratifiedShuffleSplit,drug,4.357143,0.389286
2,SVC,10,True,StratifiedKFold,0.378049,0.045303,NaN,10,0.3,StratifiedShuffleSplit,drug,3.780488,0.378049
3,SVC,10,True,StratifiedKFold,0.752500,0.077015,1:01:31,10,0.3,StratifiedShuffleSplit,stress,7.525000,0.758750
4,SVC,10,True,StratifiedKFold,0.742500,0.062300,1:08:53,10,0.3,StratifiedShuffleSplit,stress,7.425000,0.758750
5,SVC,10,True,StratifiedKFold,0.777500,0.069327,1:04:04,10,0.3,StratifiedShuffleSplit,stress,7.775000,0.758750
6,SVC,10,True,StratifiedKFold,0.762500,0.053910,0:57:07,10,0.3,StratifiedShuffleSplit,stress,7.625000,0.758750
7,SVC,10,True,StratifiedKFold,0.780000,0.103494,0:19:59,10,0.1,StratifiedShuffleSplit,stress,7.800000,0.813333
8,SVC,10,True,StratifiedKFold,0.824444,0.106782,0:33:50,30,0.1,StratifiedShuffleSplit,stress,24.733333,0.813333
9,EN,10,NaN,NaN,0.350000,0.021429,0:14:23,10,0.1,StratifiedShuffleSplit,drug,3.500000,0.341429


In [124]:
aggregated = grouped.agg(np.sum)
aggregated

results.accuracy.mean  \
classifier cross_val.n_folds split.test_size target                          
EN         10                0.1             drug                 1.045238   
                                             stress               1.140000   
                             0.3             drug                 0.326829   
SVC        10                0.1             drug                 0.809524   
                                             stress               2.417778   
                             0.3             drug                 0.378049   
                                             stress               3.035000   
           20                0.1             drug                 0.778571   
                                             stress               0.913333   

                                                     results.accuracy.std  \
classifier cross_val.n_folds split.test_size target                         
EN         10                0.1             drug                0.106854   
                                             stress              0.256576   
                             0.3             drug                0.033085   
SVC        10                0.1             drug                0.189349   
                                             stress              0.282079   
                             0.3             drug                0.045303   
                                             stress              0.262551   
           20                0.1             drug                0.291814   
                                             stress              0.060000   

                                                     split.n_iter       n*mu  
classifier cross_val.n_folds split.test_size target                           
EN         10                0.1             drug              50  17.071429  
                                             stress            50  18.866667  
                             0.3             drug              10   3.268293  
SVC        10                0.1             drug              40  15.571429  
                                             stress            50  40.666667  
                             0.3             drug              10   3.780488  
                                             stress            40  30.350000  
           20                0.1             drug              20   7.785714  
                                             stress            10   9.133333

In [125]:
aggregated['new_mu'] = aggregated['n*mu'] / aggregated['split.n_iter']
aggregated

results.accuracy.mean  \
classifier cross_val.n_folds split.test_size target                          
EN         10                0.1             drug                 1.045238   
                                             stress               1.140000   
                             0.3             drug                 0.326829   
SVC        10                0.1             drug                 0.809524   
                                             stress               2.417778   
                             0.3             drug                 0.378049   
                                             stress               3.035000   
           20                0.1             drug                 0.778571   
                                             stress               0.913333   

                                                     results.accuracy.std  \
classifier cross_val.n_folds split.test_size target                         
EN         10                0.1             drug                0.106854   
                                             stress              0.256576   
                             0.3             drug                0.033085   
SVC        10                0.1             drug                0.189349   
                                             stress              0.282079   
                             0.3             drug                0.045303   
                                             stress              0.262551   
           20                0.1             drug                0.291814   
                                             stress              0.060000   

                                                     split.n_iter       n*mu  \
classifier cross_val.n_folds split.test_size target                            
EN         10                0.1             drug              50  17.071429   
                                             stress            50  18.866667   
                             0.3             drug              10   3.268293   
SVC        10                0.1             drug              40  15.571429   
                                             stress            50  40.666667   
                             0.3             drug              10   3.780488   
                                             stress            40  30.350000   
           20                0.1             drug              20   7.785714   
                                             stress            10   9.133333   

                                                       new_mu  
classifier cross_val.n_folds split.test_size target            
EN         10                0.1             drug    0.341429  
                                             stress  0.377333  
                             0.3             drug    0.326829  
SVC        10                0.1             drug    0.389286  
                                             stress  0.813333  
                             0.3             drug    0.378049  
                                             stress  0.758750  
           20                0.1             drug    0.389286  
                                             stress  0.913333

In [128]:
results.join(aggregated['new_mu'], on=['classifier', 'cross_val.n_folds', 'split.test_size', 'target'])

,classifier,cross_val.n_folds,cross_val.shuffle,cross_val.type,results.accuracy.mean,results.accuracy.std,results.time,split.n_iter,split.test_size,split.type,target,n*mu,new_mu
0,SVC,20,True,StratifiedKFold,0.414286,0.162255,NaN,10,0.1,StratifiedShuffleSplit,drug,4.142857,0.389286
1,SVC,10,True,StratifiedKFold,0.435714,0.087190,NaN,10,0.1,StratifiedShuffleSplit,drug,4.357143,0.389286
2,SVC,10,True,StratifiedKFold,0.378049,0.045303,NaN,10,0.3,StratifiedShuffleSplit,drug,3.780488,0.378049
3,SVC,10,True,StratifiedKFold,0.752500,0.077015,1:01:31,10,0.3,StratifiedShuffleSplit,stress,7.525000,0.758750
4,SVC,10,True,StratifiedKFold,0.742500,0.062300,1:08:53,10,0.3,StratifiedShuffleSplit,stress,7.425000,0.758750
5,SVC,10,True,StratifiedKFold,0.777500,0.069327,1:04:04,10,0.3,StratifiedShuffleSplit,stress,7.775000,0.758750
6,SVC,10,True,StratifiedKFold,0.762500,0.053910,0:57:07,10,0.3,StratifiedShuffleSplit,stress,7.625000,0.758750
7,SVC,10,True,StratifiedKFold,0.780000,0.103494,0:19:59,10,0.1,StratifiedShuffleSplit,stress,7.800000,0.813333
8,SVC,10,True,StratifiedKFold,0.824444,0.106782,0:33:50,30,0.1,StratifiedShuffleSplit,stress,24.733333,0.813333
9,EN,10,NaN,NaN,0.350000,0.021429,0:14:23,10,0.1,StratifiedShuffleSplit,drug,3.500000,0.341429
